# 量化测试

sklearn的一般流程
https://blog.csdn.net/ycarry2017/article/details/85053217

iris数据集可视化
https://blog.csdn.net/weixin_39506322/article/details/82960476

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tushare as ts

In [ ]:
#df = ts.get_k_data("600050", start="1998-01-01")
df = pd.read_csv("600519.csv", index_col="date", parse_dates=["date"])
df = df[df.index.year >= 2020].copy()
df.head()

In [ ]:
df['diff_v'] =  (df['volume'] - df['volume'].shift(1)) / df['volume'].shift(1)
df.head()

In [ ]:
df['diff'] =  (df['open'] - df['close'].shift(1)) / df['close'].shift(1)*10
df['tomorrow'] = (df['diff'].shift(-1) > 0).apply(lambda x: 1 if x else -1)
df.fillna(0, inplace=True)
df[['diff', 'diff_v', 'tomorrow']].head()

In [ ]:
df[['diff','tomorrow']].plot(figsize = (12, 8))

In [ ]:
df[['diff','tomorrow']].plot(kind="scatter", x="diff", y="tomorrow")

In [ ]:
import seaborn as sns
sns.jointplot(x="diff", y="tomorrow", data=df[['diff','tomorrow']], height=5)

In [ ]:
sns.FacetGrid(df, hue="tomorrow", height=5) \
   .map(plt.scatter, "diff", "diff_v") \
   .add_legend()

In [ ]:
sns.boxplot(x="tomorrow", y="diff", data=df)

In [ ]:
ax = sns.boxplot(x="tomorrow", y="diff", data=df)
ax = sns.stripplot(x="tomorrow", y="diff", data=df, jitter=True, edgecolor="gray")

In [ ]:
sns.violinplot(x="tomorrow", y="diff", data=df, size=6)

In [ ]:
sns.FacetGrid(df, hue="tomorrow", height=6) \
   .map(sns.kdeplot, "diff") \
   .add_legend()

In [ ]:
sns.pairplot(df[['diff', 'diff_v', 'tomorrow']], hue="tomorrow", height=3)

In [ ]:
df[['diff', 'diff_v', 'tomorrow']].boxplot(by="tomorrow", figsize=(12, 6))

In [ ]:
from pandas.plotting import andrews_curves
andrews_curves(df[['diff', 'diff_v', 'tomorrow']], "tomorrow")

In [ ]:
from pandas.plotting import parallel_coordinates
parallel_coordinates(df[['diff', 'diff_v', 'tomorrow']], "tomorrow")

In [ ]:
from pandas.plotting import radviz
radviz(df[['diff', 'diff_v', 'tomorrow']], "tomorrow")

In [ ]:
X = df[['diff', 'diff_v']].values
X

In [ ]:
df['tomorrow'].value_counts()

In [ ]:
y = df['tomorrow'].values
y

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1.get the dataset
from sklearn import datasets
iris = datasets.load_iris()
#X = iris.data
#y = iris.target

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0)

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = "linear", probability = True)
classifier.fit(X_train, y_train)
# predict the result
y_pred = classifier.predict(X_test)
# 查看每种类别的概率
y_pred_proba = classifier.predict_proba(X_test)

In [ ]:
y_pred_proba

In [ ]:
y_pred

In [ ]:
params = classifier.get_params()
params

In [ ]:
score = classifier.score(X_test, y_test)
score

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(classifier ,X, y, cv = 5)
print(scores)

In [ ]:
from sklearn.model_selection import GridSearchCV
# 估计器
svc = SVC()
# 超参数空间
param_grid = [{'C':[0.1, 1, 10, 100, 1000], 'kernel':['linear']},
               {'C':[0.1, 1, 10, 100, 1000], 'kernel':['rbf'], 'gamma':[0.001, 0.01]}]
# 一共15种情况

# 评分函数
scoring = 'accuracy'

# 指定采样方法, clf即最佳模型
clf = GridSearchCV(svc, param_grid, scoring= scoring, cv = 10, iid=True) # 返回最佳模型
clf.fit(X_train, y_train)
y_pred_best = clf.predict(X_test) 

score_best = clf.score(X_test, y_pred_best)  # 100%
params_best = clf.get_params()   # 最优模型的参数
params_best

In [ ]:
import joblib
joblib.dump(clf, 'clf_joblib.pkl')  # 与pickle用法非常类似
clf3 = joblib.load('clf_joblib.pkl')
y3_pred = clf3.predict(X_test)

In [ ]:
y3_pred

In [ ]:
X_test